# Install TileDB.CSharp

In [ ]:
#r "nuget:TileDB.CSharp"

# Create an array

In [ ]:
using TileDB.CSharp;
var ctx = TileDB.CSharp.Context.GetDefault();
var dom = new TileDB.CSharp.Domain(ctx);
var dim1 = TileDB.CSharp.Dimension.Create<int>(ctx,"rows",1,4,4);
var dim2 = TileDB.CSharp.Dimension.Create<int>(ctx,"cols",1,4,4);
dom.AddDimension(dim1);
dom.AddDimension(dim2);
var schema = new TileDB.CSharp.ArraySchema(ctx,TileDB.CSharp.ArrayType.Dense);
schema.SetDomain(dom);
var attr1 = new TileDB.CSharp.Attribute(ctx,"a",TileDB.CSharp.DataType.Int32);
schema.AddAttribute(attr1);
string array_uri = "test_metadata_array";
var vfs = new TileDB.CSharp.VFS(ctx);
if(vfs.IsDir(array_uri))
{
    vfs.RemoveDir(array_uri);
}
TileDB.CSharp.Array.Create(ctx,array_uri,schema);

# Add metadata

In [ ]:
using(var array_write = new TileDB.CSharp.Array(ctx,array_uri)) 
{
    array_write.Open(TileDB.CSharp.QueryType.Write);
    var int_metadata = new int[2] {1,100};
    array_write.PutMetadata<int>("int_metadata",int_metadata);
    var double_metadata = new double[3] {1.0,100.0,1000.0};
    array_write.PutMetadata<double>("double_metadata", double_metadata);
    array_write.PutMetadata("key1","value1");
    array_write.PutMetadata("key2","value2");
    array_write.PutMetadata("key3","value3");
    array_write.Close();
}

# Get metadata

In [ ]:
var array_read = new TileDB.CSharp.Array(ctx,array_uri);
array_read.Open(TileDB.CSharp.QueryType.Read);

## Get metadata from index

In [ ]:
var metadata_num = array_read.MetadataNum();
System.Console.WriteLine("number of metadata:{0}",metadata_num);

In [ ]:
var metadata_keys = array_read.MetadataKeys();
System.Console.WriteLine("metadata keys:{0}", String.Join(" ", metadata_keys));

In [ ]:
var (key0,data0) = array_read.GetMetadataFromIndex<double>(0);
System.Console.WriteLine("key0:{0},data0:{1}", key0, String.Join(" ",data0));

## Get metadata from key

In [ ]:
var int_metadata = array_read.GetMetadata<int>("int_metadata");
System.Console.WriteLine("int_metadata:{0}", String.Join(" ", int_metadata));

In [ ]:
var double_metadata = array_read.GetMetadata<double>("double_metadata");
System.Console.WriteLine("double_metadata:{0}", String.Join(" ", double_metadata));

In [ ]:
var value3 = array_read.GetMetadata("key3");
System.Console.WriteLine("value3:{0}", value3);

In [ ]:
array_read.Close();

# Clean up

In [ ]:
if(vfs.IsDir(array_uri))
{
    vfs.RemoveDir(array_uri);
}